# IndicF5 Gradio Demo

This notebook requires a **GPU runtime**. Go to **Runtime → Change runtime type → T4 GPU** before running.

In [ ]:
# Check GPU availability - THIS MODEL REQUIRES GPU
import torch
if not torch.cuda.is_available():
    raise RuntimeError('❌ GPU not available! Go to Runtime → Change runtime type → T4 GPU')
print(f'✅ GPU available: {torch.cuda.get_device_name(0)}')

In [ ]:
# CRITICAL: Disable torch.compile/dynamo BEFORE any model loads
# This fixes the silent audio bug
import os
os.environ['TORCH_COMPILE_DISABLE'] = '1'
os.environ['TORCHDYNAMO_DISABLE'] = '1'

import torch
import torch._dynamo
torch._dynamo.config.suppress_errors = True
torch._dynamo.disable()

# Make torch.compile a no-op
original_compile = torch.compile
torch.compile = lambda model, *args, **kwargs: model

print('✅ torch.compile/dynamo disabled')

In [ ]:
# Clear HuggingFace cache
!rm -rf ~/.cache/huggingface/modules/
print('✅ HuggingFace modules cache cleared')

In [ ]:
# Clone the repository (fresh clone)
import os
!rm -rf f15
!git clone https://github.com/beginner4a3/f15
os.chdir('f15')

In [ ]:
# Install compatible library versions
%pip install -q 'transformers<4.50' accelerate
%pip install -q git+https://github.com/ai4bharat/IndicF5.git
%pip install -q gradio spaces huggingface_hub torchcodec

In [ ]:
# PATCH: Fix app.py
with open('app.py', 'r') as f:
    content = f.read()

# Disable torch.compile
content = content.replace(
    'if hasattr(torch, "compile"):',
    'if False:'
)

# Enable share=True
content = content.replace(
    'iface.launch()',
    'iface.launch(share=True, debug=True)'
)

# Fix deprecated autocast
content = content.replace(
    'torch.cuda.amp.autocast(enabled=True)',
    "torch.amp.autocast('cuda', enabled=True)"
)

with open('app.py', 'w') as f:
    f.write(content)

print('✅ app.py patched')

In [ ]:
# Login to Hugging Face (optional)
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
# IMPORTANT: Set env vars again before running
import os
os.environ['TORCH_COMPILE_DISABLE'] = '1'
os.environ['TORCHDYNAMO_DISABLE'] = '1'

# Run the app
!TORCH_COMPILE_DISABLE=1 TORCHDYNAMO_DISABLE=1 python app.py